In [3]:
# to hide tensorflow endless output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
from essentia.standard import MonoLoader, TensorflowPredictVGGish, PercivalBpmEstimator, TonalExtractor, Danceability, TensorflowPredictMusiCNN
import matplotlib.pyplot as plt
import json

In [5]:
# Convert mp3 to vector_real
filename = 'never-gonna-give-you-up.mp3'
audio = MonoLoader(filename=filename)()

# Compute BPM
bpm = PercivalBpmEstimator()(audio)

# Compute tone
ret = TonalExtractor()(audio)
key = ret[9]
scale = ret[10]

# Compute danceability
ret = Danceability()(audio)
dance = ret[0] * 10 / 3

# Compute happiness
with open('mood_happy-musicnn-mtt-2.json', 'r') as json_file:
    metadata1 = json.load(json_file)
model_h = TensorflowPredictMusiCNN(graphFilename="mood_happy-musicnn-mtt-2.pb")
activations_h = model_h(audio)

# Compute energy as: aggresive + mood_party + bpm
with open('mood_aggressive-musicnn-mtt-2.json', 'r') as json_file:
    metadata2 = json.load(json_file)
model_a = TensorflowPredictMusiCNN(graphFilename="mood_aggressive-musicnn-mtt-2.pb")
activations_a = model_a(audio)

with open('mood_party-musicnn-mtt-2.json', 'r') as json_file:
    metadata3 = json.load(json_file)
model_p = TensorflowPredictMusiCNN(graphFilename="mood_party-musicnn-mtt-2.pb")
activations_p = model_a(audio)

happy = 0
aggressive = 0
party = 0
for label, probability in zip(metadata1['classes'], activations_h.mean(axis=0)):
    if label == 'happy':
        happy = probability
        
for label, probability in zip(metadata2['classes'], activations_a.mean(axis=0)):
    if label == 'aggressive':
        aggressive = probability
        
for label, probability in zip(metadata3['classes'], activations_p.mean(axis=0)):
    if label == 'party':
        party = probability

RuntimeError: Error while configuring MonoLoader: AudioLoader: Could not open file "never-gonna-give-you-up.mp3", error = No such file or directory

In [3]:
print(f'bpm: {bpm:.0f}')
print(f'tone: {key} {scale}')
print(f'dance: {dance:.0f}')
print(f'energy: {party * aggressive * bpm / 10:.0f}')
print(f'happiness: {happy * 10:.0f}')

bpm: 114
tone: C# major
dance: 5
energy: 4
happiness: 3
